In [1]:
# Step 1: Import libraries
# ===============================
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# ===============================
# Step 2: Dataset Path
# ===============================

DATASET_SRC = '/kaggle/input/time-series-air-quality-data-of-india-2010-2023'

In [4]:
# List all files inside the dataset folder
files = os.listdir(DATASET_SRC)
print("Total files available:", len(files))
print("First 10 files:", files[:10])  # show first 10 file names


Total files available: 454
First 10 files: ['DL023.csv', 'WB005.csv', 'MH018.csv', 'UP017.csv', 'BR008.csv', 'BR003.csv', 'MP016.csv', 'KA035.csv', 'CH001.csv', 'PY001.csv']


In [5]:
# Load one sample file
sample_file = os.path.join(DATASET_SRC, "DL023.csv")
df = pd.read_csv(sample_file)

# Check shape and first few rows
print("Shape:", df.shape)
df.head()

Shape: (45230, 22)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),CO (ug/m3),...,Toluene (),Temp (degree C),RH (%),WS (m/s),WD (degree),SR (W/mt2),BP (mmHg),VWS (m/s),AT (degree C),RF (mm)
0,2018-02-01 10:00:00,2018-02-01 11:00:00,322.00,487.00,4.53,26.33,18.72,24.92,11.06,0.58,...,NaN,NaN,36.60,1.86,298.60,601.00,994.00,0.0,22.16,NaN
1,2018-02-01 11:00:00,2018-02-01 12:00:00,245.92,427.42,5.96,26.08,32.14,37.77,20.26,0.94,...,NaN,NaN,32.75,1.98,296.42,602.00,994.00,0.0,24.08,NaN
2,2018-02-01 12:00:00,2018-02-01 13:00:00,176.67,368.83,2.70,15.93,18.62,38.67,12.48,0.73,...,NaN,NaN,34.17,1.95,297.00,602.92,992.92,0.0,25.52,NaN
3,2018-02-01 13:00:00,2018-02-01 14:00:00,149.00,333.75,1.33,11.37,23.08,24.69,4.28,0.61,...,NaN,NaN,32.92,2.58,310.58,603.00,991.83,0.0,26.36,NaN
4,2018-02-01 14:00:00,2018-02-01 15:00:00,113.08,273.25,1.22,15.52,33.15,7.96,0.53,0.52,...,NaN,NaN,31.58,2.80,318.08,603.00,990.92,0.0,26.77,NaN


In [6]:
df_states = pd.read_csv(f'{DATASET_SRC}/stations_info.csv')
df_states.drop(columns=['agency', 'station_location', 'start_month'], inplace=True)
df_states.head()

,file_name,state,city,start_month_num,start_year
0,AP001,Andhra Pradesh,Tirupati,7,2016
1,AP002,Andhra Pradesh,Vijayawada,5,2017
2,AP003,Andhra Pradesh,Visakhapatnam,7,2017
3,AP004,Andhra Pradesh,Rajamahendravaram,9,2017
4,AP005,Andhra Pradesh,Amaravati,11,2017


In [7]:
unique_states = df_states['state'].unique()
unique_states

array(['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar',
       'Chhattisgarh', 'Chandigarh', 'Delhi', 'Gujarat',
       'Himachal Pradesh', 'Haryana', 'Jharkhand', 'Jammu and Kashmir',
       'Karnataka', 'Kerala', 'Maharashtra', 'Meghalaya', 'Manipur',
       'Madhya Pradesh', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab',
       'Puducherry', 'Rajasthan', 'Sikkim', 'Telangana', 'Tamil Nadu',
       'Tripura', 'Uttarakhand', 'Uttar Pradesh', 'West Bengal'],
      dtype=object)

In [8]:
def combine_state_df(state_name):
    '''
    Combine all state files into a single dataframe and attaching the city information.

    Parameters
    ----------
        state_name (str): The name of the state

    Return
    ------
        df (DataFrame): The combined dataframe from all files of a specific state
    '''
    
    state_code = df_states[df_states['state'] == state_name]['file_name'].iloc[0][:2]
    state_files = glob.glob(f'{DATASET_SRC}/{state_code}*.csv')
    print(f'Combining a total of {len(state_files)} files...\n')

    combined_df = []
    
    for state_file in state_files:
        file_name = state_file.split(f'{DATASET_SRC}/')[1][0:-4]
        file_df = pd.read_csv(state_file)
        file_df['city'] = df_states[df_states['file_name'] == file_name]['city'].values[0]
        file_df['city'] = file_df['city'].astype('string')
        combined_df.append(file_df)
        
    return pd.concat(combined_df)

In [9]:
df = combine_state_df('Delhi')
df.info()

Combining a total of 40 files...

<class 'pandas.core.frame.DataFrame'>
Index: 2796171 entries, 0 to 113961
Data columns (total 60 columns):
 #   Column               Dtype  
---  ------               -----  
 0   From Date            object 
 1   To Date              object 
 2   PM2.5 (ug/m3)        float64
 3   PM10 (ug/m3)         float64
 4   NO (ug/m3)           float64
 5   NO2 (ug/m3)          float64
 6   NOx (ppb)            float64
 7   NH3 (ug/m3)          float64
 8   SO2 (ug/m3)          float64
 9   CO (ug/m3)           float64
 10  Ozone (ug/m3)        float64
 11  Benzene (ug/m3)      float64
 12  Toluene ()           float64
 13  Temp (degree C)      float64
 14  RH (%)               float64
 15  WS (m/s)             float64
 16  WD (degree)          float64
 17  SR (W/mt2)           float64
 18  BP (mmHg)            float64
 19  VWS (m/s)            float64
 20  AT (degree C)        float64
 21  RF (mm)              float64
 22  city                 string 
 23  CO (